In [ ]:
#Basic working model using collaborative filtering with watched, this is just removing the watched movies from list and rating genres from watched history

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [ ]:

# Sample user data
user_data = {
    'id': [1, 2, 3],
    'happy_movie': [['Drama', 'Comedy'], ['Romance', 'Family'], ['Adventure']],
    'sad_movie': [['Drama', 'Thriller'], ['Crime'], ['Horror', 'Science Fiction']],
    'neutral_movie': [['Comedy'], ['Action', 'Adventure'], ['Crime']],
    'fav_movie': ['The Godfather', "Schindler's List", 'The Shawshank Redemption']
}
user = pd.DataFrame(user_data)


# Load movie data from the specified file
file_path = '/content/drive/MyDrive/final project sample data/10000 Movies Data'
minfo = pd.read_csv(file_path)
movie = minfo[['Movie_id', 'title', 'Genres', 'popularity', 'vote_average', 'vote_count']]



# Sample watched data
watched_data = {
    'userid': [1, 2, 3, 1, 2],
    'movieid': [238, 278, 240, 19404, 122],
    'rating_given': [8.0, 5.5, 9.5, 7.0, 6.5],
    'watching_mood': ['Happy', 'Sad', 'Neutral', 'Happy', 'Neutral']
}
watched = pd.DataFrame(watched_data)

In [114]:
minfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9980 entries, 0 to 9979
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    9980 non-null   int64  
 1   Movie_id      9980 non-null   int64  
 2   title         9980 non-null   object 
 3   Genres        9980 non-null   object 
 4   release_date  9980 non-null   object 
 5   Keywords      9980 non-null   object 
 6   overview      9974 non-null   object 
 7   poster_path   9980 non-null   object 
 8   Budget        9980 non-null   int64  
 9   Revenue       9980 non-null   int64  
 10  popularity    9980 non-null   float64
 11  vote_average  9980 non-null   float64
 12  vote_count    9980 non-null   int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 1013.7+ KB


In [ ]:
movie

,Movie_id,title,Genres,popularity,vote_average,vote_count
0,238,The Godfather,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",93.552,8.7,16814
1,278,The Shawshank Redemption,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",78.664,8.7,22542
2,240,The Godfather Part II,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",55.752,8.6,10187
3,19404,Dilwale Dulhania Le Jayenge,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",22.150,8.6,3927
4,424,Schindler's List,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",53.542,8.6,13382
...,...,...,...,...,...,...
9975,384737,Precious Cargo,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",14.663,5.2,243
9976,282813,The Pyramid,"[{'id': 27, 'name': 'Horror'}]",17.892,5.2,861
9977,134375,Home Alone: The Holiday Heist,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",23.978,5.2,335
9978,10477,Driven,"[{'id': 28, 'name': 'Action'}]",13.200,5.2,594


In [ ]:
# prompt: Using dataframe movie: i want Genres to be reformatted with only list of name of genres, right now it has id and name and addtional text

import ast

# Create a copy of the dataframe
movie_genres = movie.copy()

# Convert the Genres column from string to list of dictionaries
movie_genres['Genres'] = movie_genres['Genres'].apply(ast.literal_eval)

# Extract the names of the genres from the list of dictionaries
movie_genres['Genres'] = movie_genres['Genres'].apply(lambda x: [i['name'] for i in x])

# Display the first 10 rows of the dataframe
movie_genres.head(10)


,Movie_id,title,Genres,popularity,vote_average,vote_count
0,238,The Godfather,"[Drama, Crime]",93.552,8.7,16814
1,278,The Shawshank Redemption,"[Drama, Crime]",78.664,8.7,22542
2,240,The Godfather Part II,"[Drama, Crime]",55.752,8.6,10187
3,19404,Dilwale Dulhania Le Jayenge,"[Comedy, Drama, Romance]",22.150,8.6,3927
4,424,Schindler's List,"[Drama, History, War]",53.542,8.6,13382
5,129,Spirited Away,"[Animation, Family, Fantasy]",78.519,8.5,13476
6,667257,Impossible Things,"[Family, Drama]",10.676,8.5,271
7,389,12 Angry Men,[Drama],27.767,8.5,6754
8,372058,Your Name.,"[Romance, Animation, Drama]",120.614,8.5,9178
9,496243,Parasite,"[Comedy, Thriller, Drama]",61.872,8.5,14608


In [ ]:
# Assuming user, watched, and movie DataFrames are already defined as per your description.

# Merge the watched movies data with the movie details
watched_movies = pd.merge(watched, movie_genres, left_on='movieid', right_on='Movie_id')



In [ ]:
watched_movies

,userid,movieid,rating_given,watching_mood,Movie_id,title,Genres,popularity,vote_average,vote_count
0,1,238,8.0,Happy,238,The Godfather,"[Drama, Crime]",93.552,8.7,16814
1,2,278,5.5,Sad,278,The Shawshank Redemption,"[Drama, Crime]",78.664,8.7,22542
2,3,240,9.5,Neutral,240,The Godfather Part II,"[Drama, Crime]",55.752,8.6,10187
3,1,19404,7.0,Happy,19404,Dilwale Dulhania Le Jayenge,"[Comedy, Drama, Romance]",22.150,8.6,3927
4,2,122,6.5,Neutral,122,The Lord of the Rings: The Return of the King,"[Adventure, Fantasy, Action]",119.972,8.5,20423


In [ ]:

from itertools import chain

# Assuming 'user' is your DataFrame containing user preferences.
# Sample data structure for 'user' DataFrame is provided in your initial description.

# Function to expand and tag mood to genres
def expand_user_genres(user_df, mood_column):
    # Explode the DataFrame by the specified mood to handle multiple genres
    expanded_df = user_df[['id', mood_column]].explode(mood_column)
    # Assign the current mood as a new column
    expanded_df['mood'] = mood_column.split('_')[0].capitalize()
    # Rename the genre column to a generic 'genre' column
    expanded_df = expanded_df.rename(columns={mood_column: 'genre'})
    return expanded_df

# Process each mood
happy_genres = expand_user_genres(user, 'happy_movie')
sad_genres = expand_user_genres(user, 'sad_movie')
neutral_genres = expand_user_genres(user, 'neutral_movie')

# Concatenate all moods together
all_genres = pd.concat([happy_genres, sad_genres, neutral_genres])

# Optional: Reset index if you want a clean DataFrame index
all_genres.reset_index(drop=True, inplace=True)

all_genres.head()


,id,genre,mood
0,1,Drama,Happy
1,1,Comedy,Happy
2,2,Romance,Happy
3,2,Family,Happy
4,3,Adventure,Happy


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Assuming movie_genres DataFrame has a 'Genres' column with lists of genres
mlb = MultiLabelBinarizer()
movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
movie_genres_df = pd.DataFrame(movie_genres_encoded, columns=mlb.classes_, index=movie_genres['Movie_id'])



In [113]:
movie_genres_df

,Action,Adventure,Animation,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Movie_id,,,,,,,,,,,,,,,,,,
238,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
278,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
240,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
19404,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
424,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384737,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
282813,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
134375,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
# def recommend_movies_with_ratings_and_mood(user_id, current_mood, top_n=5):
#     # Get genres the user prefers in the current mood
#     mood_genres = all_genres[(all_genres['id'] == user_id) & (all_genres['mood'] == current_mood)]
#     user_pref_genres = mood_genres['genre'].unique()

#     # Get movies watched and rated by the user in the specified mood
#     mood_watched_movies = watched_movies[(watched_movies['userid'] == user_id) & (watched_movies['watching_mood'].str.capitalize() == current_mood)]

#     # Calculate average ratings for genres in this mood

#     mood_watched_movies_details = mood_watched_movies.merge(movie_genres, left_on='movieid', right_index=True)

#     # Adjusted section to use 'Genres_x'
#     genre_ratings = {}
#     for genre in user_pref_genres:
#         # Here we filter using 'Genres_x'
#         genre_movies = mood_watched_movies_details[mood_watched_movies_details['Genres_x'].apply(lambda x: genre in x)]
#         avg_rating = genre_movies['rating_given'].mean()
#         genre_ratings[genre] = avg_rating if not np.isnan(avg_rating) else 0


#     # Rank genres by average rating
#     ranked_genres = sorted(genre_ratings, key=genre_ratings.get, reverse=True)

#     # Encode user preferences weighted by average genre rating
#     user_pref_encoded = mlb.transform([ranked_genres])

#     # Calculate similarity between weighted user preferences and each movie's genres
#     similarity = cosine_similarity(user_pref_encoded, movie_genres_encoded)

#     # Exclude movies already watched
#     watched_movie_ids = watched_movies[watched_movies['userid'] == user_id]['movieid'].unique()
#     potential_movies = movie_genres[~movie_genres.index.isin(watched_movie_ids)]

#     # Get top N recommended movie IDs based on similarity scores, excluding watched movies
#     recommended_movie_ids = similarity.argsort()[0][-top_n:][::-1]

#     # Ensure correct indexing and data type consistency for recommended_movie_ids
#     recommended_movie_ids = [potential_movies.index[i] for i in recommended_movie_ids if i < len(potential_movies)]

#     # Map movie IDs back to movie titles and genres
#     recommended_movies_info = movie_genres.loc[recommended_movie_ids][['title', 'Genres']]

#     # Create a more readable output format
#     recommended_movies_output = recommended_movies_info.apply(lambda x: f"{x['title']} (Genres: {', '.join(x['Genres'])})", axis=1)

#     return recommended_movies_output.tolist()

# # Example usage
# recommended_movies = recommend_movies_with_ratings_and_mood(1, 'Happy', 5)
# for movie in recommended_movies:
#     print(movie)


In [ ]:
def recommend_movies_with_weighted_genres(user_id, current_mood, top_n=5):
    mood_genres = all_genres[(all_genres['id'] == user_id) & (all_genres['mood'] == current_mood)]
    user_pref_genres = mood_genres['genre'].unique()

    # Check for mood-specific watched movies
    mood_watched_movies = watched_movies[(watched_movies['userid'] == user_id) & (watched_movies['watching_mood'].str.capitalize() == current_mood)]

    genre_ratings = {}
    if not mood_watched_movies.empty:
        # Calculate average ratings for genres from mood-specific watched movies
        mood_watched_movies_details = mood_watched_movies.merge(movie_genres, left_on='movieid', right_index=True)
        for genre in user_pref_genres:
            genre_movies = mood_watched_movies_details[mood_watched_movies_details['Genres_x'].apply(lambda x: genre in x)]
            avg_rating = genre_movies['rating_given'].mean()
            genre_ratings[genre] = avg_rating if not np.isnan(avg_rating) else 0
        # Normalize ratings to use as weights
        max_rating = max(genre_ratings.values(), default=1)
        genre_weights = {genre: rating / max_rating for genre, rating in genre_ratings.items()}
    else:
        genre_weights = {genre: 1 for genre in user_pref_genres}  # Default weight if no ratings

    # Encode movie genres and user preferences
    mlb = MultiLabelBinarizer()
    movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
    genre_prefs_encoded = mlb.transform([user_pref_genres])

    # Calculate similarity with weighted genres
    similarity_scores = cosine_similarity(genre_prefs_encoded, movie_genres_encoded)[0]
    weighted_similarity_scores = np.zeros_like(similarity_scores)

    for i, genre in enumerate(mlb.classes_):
        if genre in genre_weights:
            weighted_similarity_scores += similarity_scores * movie_genres_encoded[:, i] * genre_weights[genre]

    # Exclude watched movies
    watched_movie_ids = watched_movies[watched_movies['userid'] == user_id]['movieid'].unique()
    unwatched_indices = [i for i, movie_id in enumerate(movie_genres.index) if movie_id not in watched_movie_ids]

    # Recommend based on weighted similarity scores
    unwatched_similarity_scores = weighted_similarity_scores[unwatched_indices]
    top_indices = np.argsort(unwatched_similarity_scores)[-top_n:][::-1]

    recommended_movie_ids = [movie_genres.index[unwatched_indices[i]] for i in top_indices]
    recommended_movies_info = movie_genres.loc[recommended_movie_ids]
    recommended_movies_output = recommended_movies_info.apply(lambda x: f"{x['title']} (Genres: {', '.join(x['Genres'])})", axis=1).tolist()

    return recommended_movies_output

# Example usage
recommended_movies = recommend_movies_with_weighted_genres(1, 'Happy', 10)
for movie in recommended_movies:
    print(movie)


The Barbarian Invasions (Genres: Comedy, Drama)
Youth (Genres: Comedy, Drama)
Swing Vote (Genres: Comedy, Drama)
Sono tornato (Genres: Comedy, Drama)
Please Stand By (Genres: Comedy, Drama)
The Terminal (Genres: Comedy, Drama)
Another Round (Genres: Comedy, Drama)
Chocolat (Genres: Comedy, Drama)
Irresistible (Genres: Comedy, Drama)
The Fundamentals of Caring (Genres: Comedy, Drama)


In [ ]:
# Example: Recommend movies for user 1 in a "Happy" mood, including genres
recommended_movies = recommend_movies_with_weighted_genres(1, 'Happy', 10)
for movie in recommended_movies:
    print(movie)

The Barbarian Invasions (Genres: Comedy, Drama)
Youth (Genres: Comedy, Drama)
Swing Vote (Genres: Comedy, Drama)
Sono tornato (Genres: Comedy, Drama)
Please Stand By (Genres: Comedy, Drama)
The Terminal (Genres: Comedy, Drama)
Another Round (Genres: Comedy, Drama)
Chocolat (Genres: Comedy, Drama)
Irresistible (Genres: Comedy, Drama)
The Fundamentals of Caring (Genres: Comedy, Drama)


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(1, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

Obsessed (Genres: Drama, Thriller)
The Glass House (Genres: Drama, Thriller)
Unknown Origins (Genres: Drama, Thriller)
New Order (Genres: Thriller, Drama)
Wake in Fright (Genres: Drama, Thriller)
The Hand that Rocks the Cradle (Genres: Drama, Thriller)
The Consequences of Love (Genres: Drama, Thriller)
Nell (Genres: Drama, Thriller)
Room (Genres: Drama, Thriller)
The Company You Keep (Genres: Drama, Thriller)


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(1, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

The ComDads (Genres: Comedy)
A Very Bad Friend (Genres: Comedy)
Picture This (Genres: Comedy)
Blended (Genres: Comedy)
I Can Quit Whenever I Want (Genres: Comedy)
Billy Madison (Genres: Comedy)
St. Vincent (Genres: Comedy)
Moonwalkers (Genres: Comedy)
The Green Butchers (Genres: Comedy)
The Beverly Hillbillies (Genres: Comedy)


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Happy', 200)
for movie in recommended_movies:
    print(movie)


A Christmas Prince: The Royal Baby (Genres: Romance, Family)
Christmas Wedding Planner (Genres: Romance, Family, TV Movie)
The Muppets Take Manhattan (Genres: Romance, Comedy, Family)
The Little Rascals (Genres: Romance, Comedy, Family)
The Man in the Moon (Genres: Drama, Family, Romance)
Barbie and the Magic of Pegasus (Genres: Animation, Family, Romance)
Steamboat Bill, Jr. (Genres: Comedy, Family, Romance)
It Takes Two (Genres: Comedy, Family, Romance)
Look Who's Talking Now! (Genres: Romance, Comedy, Family)
The Princess and the Frog (Genres: Romance, Family, Animation)
Red Shoes and the Seven Dwarfs (Genres: Animation, Romance, Family)
Three Wishes for Cinderella (Genres: Fantasy, Family, Romance)
The Holiday Calendar (Genres: Romance, Comedy, Family)
The Princess Switch: Switched Again (Genres: Family, Romance, Comedy)
A Castle for Christmas (Genres: Romance, Comedy, Family)
The Princess Diaries (Genres: Comedy, Family, Romance)
The Ultimate Gift (Genres: Romance, Drama, Family)


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

Extremely Wicked, Shockingly Evil and Vile (Genres: Crime)
Oh Mercy (Genres: Crime)
Wonderland (Genres: Crime)
Clean (Genres: Crime)
Nine Queens (Genres: Crime)
The Good Liar (Genres: Crime)
The Sicilian Clan (Genres: Crime)
The Beast (Genres: Crime)
Badlands (Genres: Drama, Crime)
The Asphalt Jungle (Genres: Crime, Drama)


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

Snake Eyes: G.I. Joe Origins (Genres: Action, Adventure)
The Quest (Genres: Action, Adventure)
Indiana Jones and the Kingdom of the Crystal Skull (Genres: Adventure, Action)
Rogue (Genres: Action, Adventure)
Mission: Impossible - Rogue Nation (Genres: Action, Adventure)
Uncharted (Genres: Action, Adventure)
Hercules (Genres: Action, Adventure)
Skyfire (Genres: Action, Adventure)
Mission: Impossible - Fallout (Genres: Action, Adventure)
Indiana Jones and the Last Crusade (Genres: Adventure, Action)


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Happy', 10)
for movie in recommended_movies:
    print(movie)

Ben 10 Alien Swarm (Genres: Adventure)
The Flight of the Phoenix (Genres: Adventure)
Belle and Sebastian (Genres: Adventure)
The Water Man (Genres: Adventure)
Young Sherlock Holmes (Genres: Adventure)
Spread Your Wings (Genres: Adventure, Family)
Nicky Larson and the Cupid's Perfume (Genres: Adventure, Comedy)
The Fox and the Child (Genres: Adventure, Family)
Dragon Blade (Genres: Adventure, History)
Swiss Family Robinson (Genres: Adventure, Family)


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

Annihilation (Genres: Science Fiction, Horror)
Tetsuo: The Iron Man (Genres: Horror, Science Fiction)
Village of the Damned (Genres: Horror, Science Fiction)
Honeymoon (Genres: Horror, Science Fiction)
Dr. Jekyll and Mr. Hyde (Genres: Horror, Science Fiction)
The Fly (Genres: Horror, Science Fiction)
The Blob (Genres: Horror, Science Fiction)
Split Second (Genres: Horror, Science Fiction)
The Curse of Frankenstein (Genres: Horror, Science Fiction)
The Lawnmower Man (Genres: Science Fiction, Horror)


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

Extremely Wicked, Shockingly Evil and Vile (Genres: Crime)
Clean (Genres: Crime)
Oh Mercy (Genres: Crime)
The Beast (Genres: Crime)
Nine Queens (Genres: Crime)
Wonderland (Genres: Crime)
The Sicilian Clan (Genres: Crime)
The Good Liar (Genres: Crime)
Le Samouraï (Genres: Crime, Thriller)
Get Carter (Genres: Crime, Thriller)


In [ ]:
user

,id,happy_movie,sad_movie,neutral_movie,fav_movie
0,1,"[Drama, Comedy]","[Drama, Thriller]",[Comedy],The Godfather
1,2,"[Romance, Family]",[Crime],"[Action, Adventure]",Schindler's List
2,3,[Adventure],"[Horror, Science Fiction]",[Crime],The Shawshank Redemption


Relevant code ends here

In [ ]:
minfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9980 entries, 0 to 9979
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    9980 non-null   int64  
 1   Movie_id      9980 non-null   int64  
 2   title         9980 non-null   object 
 3   Genres        9980 non-null   object 
 4   release_date  9980 non-null   object 
 5   Keywords      9980 non-null   object 
 6   overview      9974 non-null   object 
 7   poster_path   9980 non-null   object 
 8   Budget        9980 non-null   int64  
 9   Revenue       9980 non-null   int64  
 10  popularity    9980 non-null   float64
 11  vote_average  9980 non-null   float64
 12  vote_count    9980 non-null   int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 1013.7+ KB


In [ ]:
minfo

,Unnamed: 0,Movie_id,title,Genres,release_date,Keywords,overview,poster_path,Budget,Revenue,popularity,vote_average,vote_count
0,0,238,The Godfather,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1972-03-14,"[{'id': 131, 'name': 'italy'}, {'id': 697, 'na...","Spanning the years 1945 to 1955, a chronicle o...",/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,245066411,245066411,93.552,8.7,16814
1,1,278,The Shawshank Redemption,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1994-09-23,"[{'id': 378, 'name': 'prison'}, {'id': 417, 'n...",Framed in the 1940s for the double murder of h...,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,28341469,28341469,78.664,8.7,22542
2,2,240,The Godfather Part II,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1974-12-20,"[{'id': 131, 'name': 'italy'}, {'id': 700, 'na...",In the continuing saga of the Corleone crime f...,/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,102600000,102600000,55.752,8.6,10187
3,3,19404,Dilwale Dulhania Le Jayenge,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-10-19,[],"Raj is a rich, carefree, happy-go-lucky second...",/2CAL2433ZeIihfX1Hb2139CX0pW.jpg,100000000,100000000,22.150,8.6,3927
4,4,424,Schindler's List,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",1993-12-15,"[{'id': 818, 'name': 'based on novel or book'}...",The true story of how businessman Oskar Schind...,/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,321365567,321365567,53.542,8.6,13382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9975,9975,384737,Precious Cargo,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",2016-07-15,"[{'id': 9730, 'name': 'crime boss'}, {'id': 10...","After a botched heist, Eddie a murderous crim...",/dOQv2PDBu9bBCZRLeRp6QQYJDr3.jpg,100659,100659,14.663,5.2,243
9976,9976,282813,The Pyramid,"[{'id': 27, 'name': 'Horror'}]",2014-12-04,"[{'id': 1299, 'name': 'monster'}, {'id': 3762,...",An archaeological team attempt to unlock the s...,/joPb4PUPP7xClD6XPiQ7laBEtZf.jpg,16861137,16861137,17.892,5.2,861
9977,9977,134375,Home Alone: The Holiday Heist,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",2012-11-25,"[{'id': 207317, 'name': 'christmas'}]",8-year-old Finn is terrified to learn his fami...,/6JPrRC0JPM06y17pUXD6w1xMvKi.jpg,0,0,23.978,5.2,335
9978,9978,10477,Driven,"[{'id': 28, 'name': 'Action'}]",2001-04-27,"[{'id': 271, 'name': 'competition'}, {'id': 36...",Talented rookie race-car driver Jimmy Bly has ...,/2aN2BBAbRyfEpfxZAQY82T8zWRq.jpg,54744738,54744738,13.200,5.2,594
